In [1]:
import os
import random

import numpy as np
import pandas as pd
import scipy.sparse as sps
import matplotlib.pyplot as plt
import seaborn as snb

# Random seed for reproducibility
SEED = 42
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [2]:
from data_manager import DatasetLoader, DatasetSplitter, URMGenerator

dataset_loader = DatasetLoader()
dataset_splitter = DatasetSplitter(dataset_loader)
dataset_train, dataset_val = dataset_splitter.load_train_val()
URM_generator = URMGenerator(dataset_train, dataset_val)
URM_train, URM_val = URM_generator.generate_implicit_URM()
URM_all = URM_train + URM_val

Loading previusly generated splits...
Generating implicit URM...


In [3]:
from evaluation.evaluator import EvaluatorHoldout

evaluator = EvaluatorHoldout(URM_val, cutoff_list=[10])

In [4]:
from Recommenders.MatrixFactorization.IALSRecommenderImplicit import (
    IALSRecommenderImplicit,
)
from skopt.space import Real, Integer, Categorical
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

output_folder_path = "result_experiments/IALSRecommenderImplicit/"
recommender_class = IALSRecommenderImplicit
n_cases = 10
n_random_starts = int(n_cases * 0.3)
metric_to_optimize = "MAP"
cutoff_to_optimize = 10

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Define hyperparameters
hyperparameters_range_dictionary = {
    "num_factors": Integer(1, 200),
    "epochs": Categorical([30]),
    "alpha": Real(low=1e-3, high=50.0, prior="log-uniform"),
    "reg": Real(low=1e-5, high=1e-2, prior="log-uniform"),
}

hyperparameter_search = SearchBayesianSkopt(
    recommender_class,
    evaluator_validation=evaluator,
)

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[
        URM_train,
    ],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={},
    EARLYSTOPPING_KEYWORD_ARGS={},
)


d:\Developing\RecSys_PoliMi_challenge_2020\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
hyperparameter_search.search(
    recommender_input_args,
    hyperparameter_search_space=hyperparameters_range_dictionary,
    n_cases=n_cases,
    n_random_starts=n_random_starts,
    save_model="best",
    output_folder_path=output_folder_path,  # Where to save the results
    output_file_name_root=recommender_class.RECOMMENDER_NAME,  # How to call the files
    metric_to_optimize=metric_to_optimize,
    cutoff_to_optimize=cutoff_to_optimize,
)

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'num_factors': 136, 'epochs': 30, 'alpha': 0.0017580914002528066, 'reg': 0.00017643527285901881}


100%|██████████| 30/30 [05:26<00:00, 10.90s/it]


EvaluatorHoldout: Processed 41629 (100.0%) in 40.58 sec. Users per second: 1026
SearchBayesianSkopt: New best config found. Config 0: {'num_factors': 136, 'epochs': 30, 'alpha': 0.0017580914002528066, 'reg': 0.00017643527285901881} - results: PRECISION: 0.0266113, PRECISION_RECALL_MIN_DEN: 0.0419704, RECALL: 0.0392930, MAP: 0.0107603, MAP_MIN_DEN: 0.0164984, MRR: 0.0816937, NDCG: 0.0382631, F1: 0.0317320, HIT_RATE: 0.2045689, ARHR_ALL_HITS: 0.0932242, NOVELTY: 0.0039111, AVERAGE_POPULARITY: 0.2396484, DIVERSITY_MEAN_INTER_LIST: 0.8681330, DIVERSITY_HERFINDAHL: 0.9868112, COVERAGE_ITEM: 0.0139960, COVERAGE_ITEM_HIT: 0.0109357, ITEMS_IN_GT: 0.9925736, COVERAGE_USER: 1.0000000, COVERAGE_USER_HIT: 0.2045689, USERS_IN_GT: 1.0000000, DIVERSITY_GINI: 0.0030685, SHANNON_ENTROPY: 6.6502653, RATIO_DIVERSITY_HERFINDAHL: 0.9871370, RATIO_DIVERSITY_GINI: 0.0060319, RATIO_SHANNON_ENTROPY: 0.4928353, RATIO_AVERAGE_POPULARITY: 4.7354968, RATIO_NOVELTY: 0.2237667, 

SearchBayesianSkopt: Saving model in

100%|██████████| 30/30 [05:30<00:00, 11.03s/it]


EvaluatorHoldout: Processed 41629 (100.0%) in 32.55 sec. Users per second: 1279
SearchBayesianSkopt: New best config found. Config 1: {'num_factors': 111, 'epochs': 30, 'alpha': 2.673331669120761, 'reg': 0.0013378598915100841} - results: PRECISION: 0.0385164, PRECISION_RECALL_MIN_DEN: 0.0624529, RECALL: 0.0589176, MAP: 0.0181172, MAP_MIN_DEN: 0.0293956, MRR: 0.1334094, NDCG: 0.0615547, F1: 0.0465812, HIT_RATE: 0.2795647, ARHR_ALL_HITS: 0.1546995, NOVELTY: 0.0044136, AVERAGE_POPULARITY: 0.1096405, DIVERSITY_MEAN_INTER_LIST: 0.9737169, DIVERSITY_HERFINDAHL: 0.9973694, COVERAGE_ITEM: 0.0477823, COVERAGE_ITEM_HIT: 0.0352144, ITEMS_IN_GT: 0.9925736, COVERAGE_USER: 1.0000000, COVERAGE_USER_HIT: 0.2795647, USERS_IN_GT: 1.0000000, DIVERSITY_GINI: 0.0146936, SHANNON_ENTROPY: 8.9193424, RATIO_DIVERSITY_HERFINDAHL: 0.9976986, RATIO_DIVERSITY_GINI: 0.0288842, RATIO_SHANNON_ENTROPY: 0.6609912, RATIO_AVERAGE_POPULARITY: 2.1665162, RATIO_NOVELTY: 0.2525192, 

SearchBayesianSkopt: Saving model in resu

...

EvaluatorHoldout: Processed 41629 (100.0%) in 34.01 sec. Users per second: 1224
SearchBayesianSkopt: Config 7 is suboptimal. Config: {'num_factors': 137, 'epochs': 30, 'alpha': 0.0825502001013137, 'reg': 0.001070562697754383} - results: PRECISION: 0.0187489, PRECISION_RECALL_MIN_DEN: 0.0291000, RECALL: 0.0269791, MAP: 0.0083627, MAP_MIN_DEN: 0.0129874, MRR: 0.0667261, NDCG: 0.0288470, F1: 0.0221234, HIT_RATE: 0.1496793, ARHR_ALL_HITS: 0.0743756, NOVELTY: 0.0046043, AVERAGE_POPULARITY: 0.0698427, DIVERSITY_MEAN_INTER_LIST: 0.9488071, DIVERSITY_HERFINDAHL: 0.9948784, COVERAGE_ITEM: 0.0339087, COVERAGE_ITEM_HIT: 0.0187293, ITEMS_IN_GT: 0.9925736, COVERAGE_USER: 1.0000000, COVERAGE_USER_HIT: 0.1496793, USERS_IN_GT: 1.0000000, DIVERSITY_GINI: 0.0086710, SHANNON_ENTROPY: 8.1509705, RATIO_DIVERSITY_HERFINDAHL: 0.9952069, RATIO_DIVERSITY_GINI: 0.0170451, RATIO_SHANNON_ENTROPY: 0.6040490, RATIO_AVERAGE_POPULARITY: 1.3801051, RATIO_NOVELTY: 0.2634288, 

Iteration No: 8 ended. Search finished for

100%|██████████| 30/30 [00:05<00:00,  5.27it/s]


EvaluatorHoldout: Processed 41629 (100.0%) in 34.32 sec. Users per second: 1213
SearchBayesianSkopt: Config 8 is suboptimal. Config: {'num_factors': 2, 'epochs': 30, 'alpha': 0.007661158573126355, 'reg': 0.0002929489853362013} - results: PRECISION: 0.0206539, PRECISION_RECALL_MIN_DEN: 0.0366429, RECALL: 0.0350543, MAP: 0.0090944, MAP_MIN_DEN: 0.0160969, MRR: 0.0742334, NDCG: 0.0344375, F1: 0.0259928, HIT_RATE: 0.1669990, ARHR_ALL_HITS: 0.0819004, NOVELTY: 0.0034843, AVERAGE_POPULARITY: 0.4718285, DIVERSITY_MEAN_INTER_LIST: 0.2372593, DIVERSITY_HERFINDAHL: 0.9237254, COVERAGE_ITEM: 0.0017138, COVERAGE_ITEM_HIT: 0.0015506, ITEMS_IN_GT: 0.9925736, COVERAGE_USER: 1.0000000, COVERAGE_USER_HIT: 0.1669990, USERS_IN_GT: 1.0000000, DIVERSITY_GINI: 0.0005372, SHANNON_ENTROPY: 3.9198469, RATIO_DIVERSITY_HERFINDAHL: 0.9240303, RATIO_DIVERSITY_GINI: 0.0010560, RATIO_SHANNON_ENTROPY: 0.2904905, RATIO_AVERAGE_POPULARITY: 9.3234203, RATIO_NOVELTY: 0.1993476, 

Iteration No: 9 ended. Search finished fo

100%|██████████| 30/30 [00:03<00:00,  8.07it/s]


EvaluatorHoldout: Processed 41629 (100.0%) in 34.35 sec. Users per second: 1212
SearchBayesianSkopt: Config 9 is suboptimal. Config: {'num_factors': 1, 'epochs': 30, 'alpha': 6.118598530602453, 'reg': 0.0070037819743638905} - results: PRECISION: 0.0210694, PRECISION_RECALL_MIN_DEN: 0.0378623, RECALL: 0.0362405, MAP: 0.0094144, MAP_MIN_DEN: 0.0169275, MRR: 0.0793284, NDCG: 0.0360302, F1: 0.0266469, HIT_RATE: 0.1758870, ARHR_ALL_HITS: 0.0862674, NOVELTY: 0.0034288, AVERAGE_POPULARITY: 0.5064750, DIVERSITY_MEAN_INTER_LIST: 0.1635159, DIVERSITY_HERFINDAHL: 0.9163512, COVERAGE_ITEM: 0.0012241, COVERAGE_ITEM_HIT: 0.0011017, ITEMS_IN_GT: 0.9925736, COVERAGE_USER: 1.0000000, COVERAGE_USER_HIT: 0.1758870, USERS_IN_GT: 1.0000000, DIVERSITY_GINI: 0.0004993, SHANNON_ENTROPY: 3.6884486, RATIO_DIVERSITY_HERFINDAHL: 0.9166537, RATIO_DIVERSITY_GINI: 0.0009814, RATIO_SHANNON_ENTROPY: 0.2733421, RATIO_AVERAGE_POPULARITY: 10.0080420, RATIO_NOVELTY: 0.1961739, 

Iteration No: 10 ended. Search finished for

In [5]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path=output_folder_path)
search_metadata = data_loader.load_data(
    recommender_class.RECOMMENDER_NAME + "_metadata.zip"
)

result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

,,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
,cutoff,,,,,,,,,,,,,,,,,,,,,
0,10,0.026611,0.04197,0.039293,0.01076,0.016498,0.081694,0.038263,0.031732,0.204569,0.093224,...,1.0,0.204569,1.0,0.003068,6.650265,0.987137,0.006032,0.492835,4.735497,0.223767
1,10,0.038516,0.062453,0.058918,0.018117,0.029396,0.133409,0.061555,0.046581,0.279565,0.1547,...,1.0,0.279565,1.0,0.014694,8.919342,0.997699,0.028884,0.660991,2.166516,0.252519
2,10,0.018213,0.028201,0.026139,0.00793,0.012244,0.063314,0.02757,0.021468,0.146364,0.070533,...,1.0,0.146364,1.0,0.007746,7.944076,0.993532,0.015227,0.588717,1.421661,0.262266
3,10,0.037983,0.061181,0.057608,0.017825,0.02873,0.130861,0.060309,0.045781,0.275793,0.151909,...,1.0,0.275793,1.0,0.014139,8.86072,0.997612,0.027793,0.656647,2.143871,0.252393
4,10,0.033813,0.059132,0.057032,0.013501,0.023764,0.104969,0.052433,0.042455,0.264143,0.11879,...,1.0,0.264143,1.0,0.009585,8.095687,0.99225,0.018843,0.599952,4.58166,0.233825
5,10,0.034538,0.057543,0.054841,0.014326,0.023994,0.110549,0.052952,0.042384,0.265008,0.125534,...,1.0,0.265008,1.0,0.019293,9.200977,0.99715,0.037925,0.681862,3.098018,0.24831
6,10,0.040863,0.067054,0.063694,0.018969,0.031475,0.139025,0.065481,0.049786,0.295395,0.161764,...,1.0,0.295395,1.0,0.0231,9.57374,0.99841,0.045408,0.709487,2.211297,0.257464
7,10,0.018749,0.0291,0.026979,0.008363,0.012987,0.066726,0.028847,0.022123,0.149679,0.074376,...,1.0,0.149679,1.0,0.008671,8.150971,0.995207,0.017045,0.604049,1.380105,0.263429
8,10,0.020654,0.036643,0.035054,0.009094,0.016097,0.074233,0.034438,0.025993,0.166999,0.0819,...,1.0,0.166999,1.0,0.000537,3.919847,0.92403,0.001056,0.290491,9.32342,0.199348


In [6]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df

,num_factors,epochs,alpha,reg
0,136,30,0.001758,0.000176
1,111,30,2.673332,0.001338
2,149,30,0.06126,0.006498
3,110,30,2.363987,0.001348
4,26,30,50.0,0.000027
5,159,30,44.624998,0.000012
6,200,30,8.44781,0.00001
7,137,30,0.08255,0.001071
8,2,30,0.007661,0.000293
9,1,30,6.118599,0.007004


In [7]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

{'num_factors': 200,
 'epochs': 30,
 'alpha': 8.447810208378677,
 'reg': 1.000821776801401e-05}

In [8]:
exception_list = search_metadata["exception_list"]
exception_list

[None, None, None, None, None, None, None, None, None, None]

In [9]:
recommender = recommender_class(URM_all)
recommender.fit(**best_hyperparameters)
recommender.save_model(
    folder_path=output_folder_path,
    file_name=recommender_class.RECOMMENDER_NAME
    + "_best_model_trained_on_everything.zip",
)

100%|██████████| 30/30 [05:42<00:00, 11.42s/it]


IALSRecommenderImplicit: Saving model in file 'result_experiments/IALSRecommenderImplicit/IALSRecommenderImplicit_best_model_trained_on_everything.zip'
IALSRecommenderImplicit: Saving complete


In [11]:
from utils.create_submission import create_submission

create_submission(recommender)

Generating recomendations...


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices